## 1. Load Data

In [2]:
import json

with open('/home/duong/Documents/project_mlops1/test/luật_đất_đai_2024.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

processed_documents = []
for item in json_data:
    combined_text = f"Trích dẫn ở: {item['title']} \n Nội dung như sau: {item['context']}"
    processed_documents.append(combined_text)


In [3]:
processed_documents

['Trích dẫn ở: luật đất đai 2024\nChương I\nQUY ĐỊNH CHUNG\nĐiều 1.Phạm vi điều chỉnh \n Nội dung như sau: Luật này quy định về chế độ sở hữu đất đai, quyền hạn và trách nhiệm của Nhà nước đại diện chủ sở hữu toàn dân về đất đai và thống nhất quản lý về đất đai, chế độ quản lý và sử dụng đất đai, quyền và nghĩa vụ của công dân, người sử dụng đất đối với đất đai thuộc lãnh thổ của nước Cộng hòa xã hội chủ nghĩa Việt Nam.',
 'Trích dẫn ở: luật đất đai 2024\nChương I\nQUY ĐỊNH CHUNG\nĐiều 2.Đối tượng áp dụng \n Nội dung như sau: 1. Cơ quan nhà nước thực hiện quyền hạn và trách nhiệm đại diện chủ sở hữu toàn dân về đất đai, thực hiện nhiệm vụ thống nhất quản lý nhà nước về đất đai. 2. Người sử dụng đất. 3. Các đối tượng khác có liên quan đến việc quản lý, sử dụng đất đai.',
 'Trích dẫn ở: luật đất đai 2024\nChương I\nQUY ĐỊNH CHUNG\nĐiều 3.Giải thích từ ngữ \n Nội dung như sau: Trong Luật này, các từ ngữ dưới đây được hiểu như sau: 1. Bản đồ địa chính là bản đồ thể hiện các thửa đất và các

## 2. Ingestion

In [11]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import time
import weaviate

client = weaviate.Client("http://35.247.178.250:85")
# Tokenize các câu
tokenizer_sent = [tokenize(sent) for sent in processed_documents]

model = SentenceTransformer('dangvantuan/vietnamese-embedding', device='cpu')

def vectorize_documents(documents):
    before = time.time()
    # Tính toán embeddings cho tất cả các câu đã tokenize
    embeddings = model.encode(documents)
    print(embeddings)
    
    after = time.time()
    print("Vectorized {} items in {}s".format(len(embeddings), after - before))
    
    return embeddings

def init_weaviate_schema(client):
    # a simple schema containing just a single class for our posts
    schema = {
        "classes": [{
            "class": "Document",
            "vectorizer": "none", # explicitly tell Weaviate not to vectorize anything, we are providing the vectors ourselves through our BERT model
            "properties": [{
                "name": "content",
                "dataType": ["text"],
            }]
        }]
    }

    # cleanup from previous runs
    client.schema.delete_all()

    client.schema.create(schema)

def import_documents_with_vectors(documents, vectors, client):
    if len(documents) != len(vectors):
        raise Exception("len of documents ({}) and vectors ({}) does not match".format(len(documents), len(vectors)))
        
    for i, document in enumerate(documents):
        try:
            client.data_object.create(
                data_object={"content": document},
                class_name='Document',
                vector=vectors[i]
            )
        except Exception as e:
            print(f"Error importing document {i}: {e}")

# Initialize Weaviate schema
init_weaviate_schema(client)

# Vectorize documents using the updated function
vectors = vectorize_documents(tokenizer_sent)

# Import documents along with their vectors into Weaviate
import_documents_with_vectors(processed_documents, vectors, client)


[[ 0.03942457 -0.21964476 -0.45683962 ...  0.3865706  -0.02393553
   0.06712171]
 [-0.00966421 -0.21868284 -0.61350703 ...  0.4141443  -0.05549915
  -0.08882433]
 [ 0.06744865 -0.40885392 -0.05951843 ...  0.28971893 -0.13678224
  -0.21996067]
 ...
 [-0.16746862  0.05598827 -0.05234704 ...  0.4200702   0.05501137
  -0.03588456]
 [-0.09190563  0.09284385 -0.15199335 ...  0.3880283  -0.10792828
   0.05973756]
 [-0.30622023 -0.13096473 -0.07144921 ...  0.4136018  -0.11419284
   0.14207299]]
Vectorized 1217 items in 53.74102473258972s


In [3]:
import weaviate

client = weaviate.Client("http://35.240.249.218:85")
schema = client.schema.get()
print(schema)


{'classes': [{'class': 'Document', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'multiTenancyConfig': {'autoTenantActivation': False, 'autoTenantCreation': False, 'enabled': False}, 'properties': [{'dataType': ['text'], 'indexFilterable': True, 'indexRangeFilters': False, 'indexSearchable': True, 'name': 'content', 'tokenization': 'word'}], 'replicationConfig': {'asyncEnabled': False, 'factor': 1}, 'shardingConfig': {'actualCount': 1, 'actualVirtualCount': 128, 'desiredCount': 1, 'desiredVirtualCount': 128, 'function': 'murmur3', 'key': '_id', 'strategy': 'hash', 'virtualPerPhysical': 128}, 'vectorIndexConfig': {'bq': {'enabled': False}, 'cleanupIntervalSeconds': 300, 'distance': 'cosine', 'dynamicEfFactor': 8, 'dynamicEfMax': 500, 'dynamicEfMin': 100, 'ef': -1, 'efConstruction': 128, 'flatSearchCutoff': 40000, 'maxConnections': 32, 'pq': {'bitCompression': False, 'centroids': 

In [31]:
client.schema.delete_all()

In [4]:
result = client.data_object.get(class_name="Document")
print(result)


{'deprecations': [], 'objects': [{'class': 'Document', 'creationTimeUnix': 1724473153547, 'id': '028246c9-1f75-4450-a9f6-d7b0e0a6a5b2', 'lastUpdateTimeUnix': 1724473153547, 'properties': {'content': 'Trích dẫn ở: Document Title\nChương V\nNGƯỜI ĐIỀU KHIỂN PHƯƠNG TIỆN\nĐiều 59.Giấy phép lái xe \n Nội dung như sau: đã có giấy phép lái xe hạng C để lái các loại xe quy định cho hạng C khi kéo rơ moóc, đầu kéo kéo sơ mi rơ moóc. 5. Giấy phép lái xe có giá trị sử dụng trong phạm vi lãnh thổ Việt Nam và lãnh thổ của nước hoặc vùng lãnh thổ mà Việt Nam ký cam kết công nhận giấy phép lái xe của nhau.'}, 'vectorWeights': None}, {'class': 'Document', 'creationTimeUnix': 1724473152342, 'id': '07867a60-0f27-40c6-b388-61233bf8a316', 'lastUpdateTimeUnix': 1724473152342, 'properties': {'content': 'Trích dẫn ở: Document Title\nChương I\nNHỮNG QUY ĐỊNH CHUNG\nĐiều 8.Các hành vi bị nghiêm cấm \n Nội dung như sau: chuyển trái phép hoặc không thực hiện đầy đủ các quy định về vận chuyển hàng nguy hiểm, động

## 3. Retrival


In [21]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import weaviate

model = SentenceTransformer('dangvantuan/vietnamese-embedding', device='cpu')

client = weaviate.Client("http://127.0.0.1:80")

def search(query="", limit=3):
    tokenized_query = tokenize(query)

    query_vector = model.encode([tokenized_query])[0]

    near_vec = {"vector": query_vector}
    
    res = client \
        .query.get("Document", ["content", "_additional {certainty}"]) \
        .with_near_vector(near_vec) \
        .with_limit(limit) \
        .do()

    for document in res["data"]["Get"]["Document"]:
        print("{:.4f}: {}".format(document["_additional"]["certainty"], document["content"]))
        print("__________-")

search("cho tôi biết thông tin về quỹ phát triển đất", 3)


0.8199: Trích dẫn ở: luật đất đai 2024
Chương VIII
PHÁT TRIỂN, QUẢN LÝ VÀ KHAI THÁC QUỸ ĐẤT
Điều 115.Tổ chức phát triển quỹ đất 
 Nội dung như sau: 1. Tổ chức phát triển quỹ đất được thành lập để tạo lập, phát triển, quản lý, khai thác quỹ đất tại địa phương. 2. Nguồn tài chính của tổ chức phát triển quỹ đất được phân bổ từ ngân sách nhà nước và ứng vốn từ quỹ phát triển đất. 3. Chính phủ quy định việc thành lập, chức năng, nhiệm vụ, tổ chức bộ máy, cơ chế quản lý, hoạt động, cơ chế tài chính của tổ chức phát triển quỹ đất.
__________-
0.8092: Trích dẫn ở: luật đất đai 2024
Chương IV
ĐỊA GIỚI ĐƠN VỊ HÀNH CHÍNH, ĐIỀU TRA CƠ BẢN VỀ ĐẤT ĐAI
Điều 51.Nguyên tắc điều tra, đánh giá đất đai và bảo vệ, cải tạo, phục hồi đất 
 Nội dung như sau: cầu thông tin đất đai cho các hoạt động kinh tế, xã hội, quốc phòng, an ninh, nghiên cứu khoa học, giáo dục và đào tạo và các nhu cầu khác của Nhà nước và xã hội.
__________-
0.8041: Trích dẫn ở: luật đất đai 2024
Chương VIII
PHÁT TRIỂN, QUẢN LÝ VÀ KHAI T

In [8]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize

tokenizer_sent = [tokenize(sent) for sent in processed_documents]

model = SentenceTransformer('dangvantuan/vietnamese-embedding',device='cpu')
embeddings = model.encode(tokenizer_sent)
print(embeddings)

[[ 0.03942457 -0.21964476 -0.45683962 ...  0.3865706  -0.02393553
   0.06712171]
 [-0.00966421 -0.21868284 -0.61350703 ...  0.4141443  -0.05549915
  -0.08882433]
 [ 0.06744865 -0.40885392 -0.05951843 ...  0.28971893 -0.13678224
  -0.21996067]
 ...
 [-0.16746862  0.05598827 -0.05234704 ...  0.4200702   0.05501137
  -0.03588456]
 [-0.09190563  0.09284385 -0.15199335 ...  0.3880283  -0.10792828
   0.05973756]
 [-0.30622023 -0.13096473 -0.07144921 ...  0.4136018  -0.11419284
   0.14207299]]


## 4. LLM

In [21]:
import requests
from llama_index.core import PromptTemplate

context_str = """
Ông Trump kể: “Tôi nghe thấy một âm thanh vo ve lớn và cảm thấy có thứ gì đó đập rất mạnh vào tai phải. Tôi tự nhủ, ôi cái gì thế nhỉ? Nó chỉ có thể là một viên đạn thôi. Bàn tay của tôi đầy máu, có máu khắp nơi".
Ông Trump kể lại trong một cuộc trả lời phỏng vấn báo Washington Examiner ngày 14-7-2024 rằng ông đã bị một viên đạn sượt qua, đó có thể là một đòn chí mạng vào hộp sọ nếu cựu tổng thống không ngoảnh đầu đi.
"Điều đáng kinh ngạc nhất là tôi không chỉ quay đầu mà còn quay đầu đúng lúc và vừa đủ" – ông Trump nói với phóng viên Byron York của Washington Examiner khi chuẩn bị tới TP Milwaukee tham dự đại hội toàn quốc của Đảng Cộng hòa.
"""

query_str = "Donal Trump là ai, cho tôi tin tức mới nhất về ông ấy, ngày sinh nữa"

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

messages = qa_template.format_messages(context_str=context_str, query_str=query_str)
prompt = messages[0].content

api_url = "http://localhost:8080/generate"
headers = {
    "Content-Type": "application/json",
}

data = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 200, 
    },
}

response = requests.post(api_url, headers=headers, json=data)


response_json = response.json()
print(response_json)


{'generated_text': '\nTrả lời: Donald Trump là cựu Tổng thống thứ 45 của Hoa Kỳ. Tin tức mới nhất về ông là ông đã bị một viên đạn sượt qua trong một vụ ám sát hụt vào năm 2024. Ngày sinh của ông là ngày 14 tháng 6 năm 1946. '}
